In [1]:
import pandas as pd
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset = dataset.drop(['User ID'],axis=1)
dataset.columns

#independent, dependent variable
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent = dataset[['Purchased']]

#split training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

#preprocessing the training and test set
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
grid_param = {
    'criterion':['gini', 'entropy', 'log_loss'],
    'max_features':['sqrt', 'log2', None]
}
grid = GridSearchCV(RandomForestClassifier(), grid_param, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None]},
             scoring='f1_weighted', verbose=3)

In [33]:
grid_pred = grid.predict(X_test)

In [35]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, grid_pred)

In [37]:
from sklearn.metrics import classification_report
cf_report = classification_report(y_test, grid_pred)

In [39]:
print(cf_matrix)
print(cf_report)

[[74  5]
 [ 5 36]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        79
           1       0.88      0.88      0.88        41

    accuracy                           0.92       120
   macro avg       0.91      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [41]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(X_test)[:,1])

0.9638777400432231

In [43]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_pred, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'log_loss', 'max_features': 'sqrt'}: 0.9166666666666666


In [45]:
re = grid.cv_results_
table = pd.DataFrame.from_dict(re)

In [47]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.339108,0.004299,0.025382,0.005761,gini,sqrt,"{'criterion': 'gini', 'max_features': 'sqrt'}",0.874254,0.875644,0.841398,0.929144,0.909115,0.885911,0.030431,5
1,0.342527,0.002849,0.022034,0.004092,gini,log2,"{'criterion': 'gini', 'max_features': 'log2'}",0.874254,0.858503,0.841398,0.911105,0.964286,0.889909,0.043740,2
2,0.326038,0.011770,0.024230,0.004236,gini,None,"{'criterion': 'gini', 'max_features': None}",0.855314,0.821429,0.859435,0.875548,0.927778,0.867901,0.034738,9
3,0.347856,0.004474,0.022899,0.003930,entropy,sqrt,"{'criterion': 'entropy', 'max_features': 'sqrt'}",0.874254,0.858503,0.841398,0.911105,0.963889,0.889830,0.043605,3
4,0.345615,0.004699,0.027047,0.004582,entropy,log2,"{'criterion': 'entropy', 'max_features': 'log2'}",0.874254,0.858503,0.823129,0.911105,0.964286,0.886255,0.048178,4
5,0.351599,0.004662,0.020866,0.000737,entropy,None,"{'criterion': 'entropy', 'max_features': None}",0.855314,0.858503,0.841398,0.875548,0.927778,0.871708,0.030068,7
6,0.348198,0.006399,0.024793,0.004571,log_loss,sqrt,"{'criterion': 'log_loss', 'max_features': 'sqrt'}",0.874254,0.875644,0.859435,0.911105,0.945469,0.893181,0.031177,1
7,0.308031,0.025846,0.020536,0.006843,log_loss,log2,"{'criterion': 'log_loss', 'max_features': 'log2'}",0.874254,0.857143,0.823129,0.892857,0.964286,0.882334,0.046972,6
8,0.302761,0.014397,0.015922,0.003680,log_loss,None,"{'criterion': 'log_loss', 'max_features': None}",0.835985,0.857143,0.841398,0.892857,0.927778,0.871032,0.034629,8
